<a href="https://colab.research.google.com/github/FabrizioBettetti/Computer_science/blob/main/Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Node(object):
  def __init__(self, name):
    """name: string"""
    self.name = name

  def getName(self):
    return self.name

  def __str__(self):
    return self.name

In [2]:
class Edge(object):
  def __init__(self, src, dest):
    """src and dest: instances of class Node"""
    self.src = src
    self.dest = dest

  def getSource(self):
    return self.src

  def getDestination(self):
    return self.dest

  def __str__(self):
    return self.src.getName() + ' -> ' + self.dest.getName()

In [3]:
class Digraph(object):
  """edges is a dict mapping each node to a list of its children"""

  def __init__(self):
    self.edges = {}

  def addNode(self, node):
    """node: instance of class Node"""
    if node in self.edges:
      raise ValueError('Duplicate node')
    else:
      self.edges[node] = []

  def addEdge(self, edge):
    """edge: instance of class Edge"""
    src = edge.getSource()
    dest = edge.getDestination()
    if not (src in self.edges and dest in self.edges):
      raise ValueError('Node not in graph')
    self.edges[src].append(dest)

  def childrenOf(self, node):
    return self.edges[node]

  def hasNode(self, node):
    return node in self.edges

  def getNode(self, name):
    for n in self.edges:
      if n.getName() == name:
        return n
    raise NameError(name)

  def __str__(self):
    result = ''
    for src in self.edges:
      for dest in self.edges[src]:
        result = result + src.getName() + ' -> ' + dest.getName() + '\n'
    return result[:-1]   #Omit final newline

In [4]:
class Graph(Digraph):
  def addEdge(self, edge):
    Digraph.addEdge(self, edge)
    rev = Edge(edge.getDestination(), edge.getSource())
    Digraph.addEdge(self, rev)

#Shortest path

In [5]:
def buildCityGraph():
  g = Digraph()

  #Create nodes
  for name in ('Boston', 'Providence', 'New York', 'Chicago', 'Denver', 'Phoenix', 'Los Angeles'):
    g.addNode(Node(name))

  #Create edges
  g.addEdge(Edge(g.getNode('Boston'), g.getNode('Providence')))
  g.addEdge(Edge(g.getNode('Boston'), g.getNode('New York')))
  g.addEdge(Edge(g.getNode('Providence'), g.getNode('Boston')))
  g.addEdge(Edge(g.getNode('Providence'), g.getNode('New York')))
  g.addEdge(Edge(g.getNode('New York'), g.getNode('Chicago')))
  g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Denver')))
  g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Phoenix')))
  g.addEdge(Edge(g.getNode('Denver'), g.getNode('Phoenix')))
  g.addEdge(Edge(g.getNode('Denver'), g.getNode('New York')))
  g.addEdge(Edge(g.getNode('Los Angeles'), g.getNode('Boston')))

  return g

##Depth first search (DFS)

In [6]:
def printPath(path):
  """path: list of nodes"""

  result = ''
  for i in range(len(path)):
    result = result + str(path[i])
    if i != len(path) - 1:
      result = result + ' -> '
  return result

In [7]:
def DFS(graph, start, end, path, shortest, toPrint=False):
  """
  start: start node.
  end: end node:
  path: list of nodes visited.
  shortest: shortest path from start to end
  """

  path = path + [start]   #path is initially an empty list. Add start node to path
  if toPrint:
    print('Current DFS path:', printPath(path))
  if start == end:   #If start node is equal to end node, done!
    return path
  for node in graph.childrenOf(start):   #If not done, consider all the children of the start node (nodes I can reach with a single edge from start node)
    if node not in path:   #Avoid cycles
      if shortest == None or len(path) < len(shortest):   #If I haven't a solution yet or current solution is not the shortest
        newPath = DFS(graph, node, end, path, shortest, toPrint)
        if newPath != None:
          shortest = newPath
    elif toPrint:
      print('Already visited', node)
  return shortest

In [8]:
def shortestPath(graph, start, end, dfs=True, toPrint=False):
  if dfs:
    return DFS(graph, start, end, [], None, toPrint)
  return BFS(graph, start, end, toPrint)

In [9]:
def testSP(source, destination, dfs=True):
  g = buildCityGraph()
  sp = shortestPath(g, g.getNode(source), g.getNode(destination), dfs=dfs, toPrint=True)
  if sp != None:
    print('Shortest path from', source, 'to', destination, 'is', printPath(sp))
  else:
    print('There is no path from', source, 'to', destination)

In [10]:
testSP('Chicago', 'Boston')

Current DFS path: Chicago
Current DFS path: Chicago -> Denver
Current DFS path: Chicago -> Denver -> Phoenix
Current DFS path: Chicago -> Denver -> New York
Already visited Chicago
Current DFS path: Chicago -> Phoenix
There is no path from Chicago to Boston


In [11]:
testSP('Boston', 'Phoenix')

Current DFS path: Boston
Current DFS path: Boston -> Providence
Already visited Boston
Current DFS path: Boston -> Providence -> New York
Current DFS path: Boston -> Providence -> New York -> Chicago
Current DFS path: Boston -> Providence -> New York -> Chicago -> Denver
Current DFS path: Boston -> Providence -> New York -> Chicago -> Denver -> Phoenix
Already visited New York
Current DFS path: Boston -> Providence -> New York -> Chicago -> Phoenix
Current DFS path: Boston -> New York
Current DFS path: Boston -> New York -> Chicago
Current DFS path: Boston -> New York -> Chicago -> Denver
Current DFS path: Boston -> New York -> Chicago -> Denver -> Phoenix
Already visited New York
Current DFS path: Boston -> New York -> Chicago -> Phoenix
Shortest path from Boston to Phoenix is Boston -> New York -> Chicago -> Phoenix


##Breadth first search (BFS)

In [12]:
def BFS(graph, start, end, toPrint=False):
  initPath = [start]
  pathQueue = [initPath]   #List of paths (list of lists)
  while len(pathQueue) != 0:
    #Get and remove oldest path in pathQueue (the path at the beginning)
    tmpPath = pathQueue.pop(0)
    if toPrint:
      print('Current BFS path:', printPath(tmpPath))
    lastNode = tmpPath[-1]
    if lastNode == end:
      return tmpPath
    for nextNode in graph.childrenOf(lastNode):
      if nextNode not in tmpPath:
        newPath = tmpPath + [nextNode]
        pathQueue.append(newPath)
  return None

In [13]:
testSP('Boston', 'Phoenix', False)

Current BFS path: Boston
Current BFS path: Boston -> Providence
Current BFS path: Boston -> New York
Current BFS path: Boston -> Providence -> New York
Current BFS path: Boston -> New York -> Chicago
Current BFS path: Boston -> Providence -> New York -> Chicago
Current BFS path: Boston -> New York -> Chicago -> Denver
Current BFS path: Boston -> New York -> Chicago -> Phoenix
Shortest path from Boston to Phoenix is Boston -> New York -> Chicago -> Phoenix
